In [1]:
from preprocess import dataset,data_deal
from sklearn.preprocessing import MinMaxScaler
from models import AE_MLP,get_models
from torch.utils.data import DataLoader,Dataset
from sklearn.model_selection import KFold
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import json
import random
from TrainandTest import train_MLP,test_MLP
import pandas as pd


def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

# Config

In [2]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
file = './data/total 12 data.xlsx'
data = pd.read_excel(file,engine="openpyxl")
data = np.array(data)
l = len(data)
para_path = './save/parameter'
batch_size = 300
plot = True
np.set_printoptions(threshold=np.sys.maxsize)
times = 0
best_seed1= best_seed2=0
best_rmse = rmse = 5

# Train

In [3]:
with open('config.json','r',encoding='utf-8')as f:
    config_list = json.load(f)
flag =  0
config = config_list[0]
Type = config['Type']
N_size = config['N_size']
lr = config['lr']
k = config['k']
alpha = config['alpha']
input_size = config['input_size']
epsilon = config['epsilon']
seed1,seed2 = config['seed1'],config['seed2']
while rmse >=0.1574:
    rmse = 0
    total_loss = 0

    seed1 = random.randint(0, 4294967295)
    seed2 = random.randint(0, 4294967295)

    kf = KFold(n_splits=k ,shuffle=True,random_state=seed1)

    total_data = dataset(data,Type,N_size,input_size)
    total_data = np.array(total_data,dtype = object)

    X = total_data[:,0]
    y = total_data[:,1].astype(np.float32)

    temp_loss = 0
    l1= X[0].shape[0]
    l2 =len(X)
    results_df = pd.DataFrame()
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        same_seeds(seed2)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = get_models(config['model'],input_size,epsilon)
        scaler = MinMaxScaler()
        X_train,X_test,y_train,y_test,scaler = data_deal(X_train,X_test,y_train,y_test,scaler,input_size,flag)
        train_MLP(X_train,y_train,model,lr,alpha,fold,para_path,flag)
        loss,results_df = test_MLP(X_test,y_test,model,fold,para_path,scaler,results_df,l1,flag)
        total_loss += loss
        temp_loss += loss

    rmse = (temp_loss/l2)**0.5
    rmse = float(rmse)
    times += 1
    
    if best_rmse>=rmse:
        best_rmse=rmse
        best_seed1=seed1
        best_seed2=seed2
        
    print(f'Times:{times},RMSE:{rmse:.10f},seed1:{seed1},seed2:{seed2},{best_rmse},{best_seed1},{best_seed2}')


Times:1,RMSE:0.2271565944,seed1:1004202784,seed2:2737134381,0.2271565943956375,1004202784,2737134381


RuntimeError: PytorchStreamReader failed reading file data/9: file read failed